In [ ]:
import MeCab
m = MeCab.Tagger('-d /usr/lib/mecab/dic/mecab-ipadic-neologd')

In [ ]:
import pandas as pd

csv_df = pd.read_csv('example.csv',encoding="utf8") 
len(csv_df)
csv_df.info()
csv_df.head()

In [ ]:
import jaconv
wakatis = []
for w in csv_df['歳出事業']:
    words1 = []
    w = jaconv.z2h(w, digit=True, ascii=True) # 半角、小文字に統一
    for c in m.parse(w).splitlines()[:-1]:  #事業名を分割し解析結果を配列にして繰り返す
        surface, feature = c.split('\t')    #分割された事業名と結果をそれぞれ格納 
        if '名詞' in feature:             #名詞ならそれをwords1に追加
            words1.append(surface)
            
    str1 = ' '.join(words1) #分割された事業名を空白で区切る
    wakatis.append(str1)    #区切られた事業名を配列に格納
# wakatis
csv_df['wakati'] = wakatis
learn_df = csv_df[['歳出事業コード','歳出事業','wakati']]
learn_df.info()
learn_df.head()

In [ ]:
def find_similarity(x, y):
        
    d0 = []
    for n in y:
        w = []
        for word in x:
            if word in n:
                w.append(word)
        d0.append(w)
    
    d1 = []    
    for v in d0:
        a = 1
        point = 0
        for word2 in v:
            for list_w in d0:
                if word2 in list_w:
                    a += 1
        
            point += len(word2)/a
        
        d1.append(point)
        
    return d1

In [ ]:
import numpy as np
import time

while True:
    
    word = input('検索:')
    if word == '終了':
        break
    
    else:
        
        t1 = time.time()
        words2 = []
        wakatis2 = []
        for c in m.parse(word).splitlines()[:-1]:
            surface, feature = c.split('\t')     
            if '名詞' in feature:
                words2.append(surface)
        str2 = ' '.join(words2) 
        wakatis2.append(str2)
        
        d = find_similarity(wakatis2[0], wakatis)
        d = np.array(d)
        topn_indices = np.argsort(d)[::-1][0:10]
        b = True
        for sim, tweet in zip(d[topn_indices], np.array(csv_df['歳出事業'])[topn_indices]):
            if sim != 0:
                print("({:}): {}".format(sim, "".join(tweet.split())))
                b = False
            if b:
                print('検索結果なし')
                break
        print('検索時間:', time.time()-t1)